## SQL (SQLAlchemy): SELECT FROM, WHERE, ORDER BY

In [8]:
import pandas as pd
from sqlalchemy import create_engine, text

# 1. Create the engine (This creates the 'geophysics.db' file automatically)
engine = create_engine('sqlite:///geophysics.db')

# 2. Define your SQL commands as strings
setup_sql = """
CREATE TABLE IF NOT EXISTS seismic_readings (
    reading_id INTEGER PRIMARY KEY,
    borehole_id INTEGER,
    magnitude REAL,
    depth_km REAL
);

-- Note: We use 'INSERT OR IGNORE' so you can run this cell multiple 
-- times without getting "Duplicate ID" errors.
INSERT OR IGNORE INTO seismic_readings (reading_id, borehole_id, magnitude, depth_km) VALUES 
(101, 1, 3.2, 5.5),
(102, 1, 4.1, 12.3),
(103, 2, 2.5, 4.1),
(104, 3, 5.0, 15.2),
(105, 1, 3.8, 8.9);
"""

query_sql = """
SELECT * FROM seismic_readings 
WHERE magnitude > 3.0 
ORDER BY depth_km DESC;
"""

# 3. Execute the Setup (Create & Insert)
with engine.connect() as conn:
    # We split by ';' to execute multiple statements if needed
    for statement in setup_sql.split(';'):
        if statement.strip():
            conn.execute(text(statement))
            conn.commit() # Important: This saves the changes
            
# Instead of asking Pandas to handle the connection, we will fetch the 
# data ourselves using SQLAlchemy and then hand the raw list of data to Pandas.
# 4. Execute the Query manually
with engine.connect() as conn:
    result = conn.execute(text(query_sql))
    # Fetch all rows into a list of dictionaries
    data = result.mappings().all()

# 5. Create the DataFrame and Display
df = pd.DataFrame(data)

if not df.empty:
    print("Filtered Seismic Readings (Magnitude > 3.0):")
    display(df) # Using display() is better for Jupyter than print()
else:
    print("No readings found matching the criteria.")

Filtered Seismic Readings (Magnitude > 3.0):


,borehole_id,depth_km,magnitude,reading_id
0,3,15.2,5.0,104
1,1,12.3,4.1,102
2,1,8.9,3.8,105
3,1,5.5,3.2,101


In [9]:
desired_order = ['depth_km', 'borehole_id', 'magnitude', 'reading_id']
df = df[desired_order]
df

,depth_km,borehole_id,magnitude,reading_id
0,15.2,3,5.0,104
1,12.3,1,4.1,102
2,8.9,1,3.8,105
3,5.5,1,3.2,101


In [35]:
# Function to quickly view a table
def view_table_0(table_name, engine):
    with engine.connect() as conn:
        result = conn.execute(text(f"SELECT * FROM {table_name}"))
        df = pd.DataFrame(result.mappings().all())
    return df

# Display both tables
print("--- BOREHOLES TABLE ---")
display(view_table_0("seismic_readings", engine))

print("Filtered Seismic Readings (Magnitude > 3.0)")
print("ORDER BY depth_km DESC:")
df

--- BOREHOLES TABLE ---


,borehole_id,depth_km,magnitude,reading_id
0,1,5.5,3.2,101
1,1,12.3,4.1,102
2,2,4.1,2.5,103
3,3,15.2,5.0,104
4,1,8.9,3.8,105


Filtered Seismic Readings (Magnitude > 3.0)
ORDER BY depth_km DESC:


,depth_km,borehole_id,magnitude,reading_id
0,15.2,3,5.0,104
1,12.3,1,4.1,102
2,8.9,1,3.8,105
3,5.5,1,3.2,101


## SQL (SQLAlchemy): FROM, JOIN, ORDER BY

In [25]:
import pandas as pd
from sqlalchemy import create_engine, text

# 1. Setup the Engine
engine = create_engine('sqlite:///geophysics.db')

# 2. Define the Full Schema and Data
# We use a list of statements to avoid execution errors
setup_statements = [
    "DROP TABLE IF EXISTS seismic_readings;", # delete old tables with that name
    "DROP TABLE IF EXISTS boreholes;",
    
    """CREATE TABLE boreholes (
        id INTEGER PRIMARY KEY,
        name TEXT, 
        formation TEXT
    );""",
    
    """CREATE TABLE seismic_readings (
        reading_id INTEGER PRIMARY KEY,
        borehole_id INTEGER REFERENCES boreholes(id),
        magnitude REAL,
        depth_km REAL
    );""",
    
    "INSERT INTO boreholes VALUES (1, 'Alpha-01', 'Basalt');",
    "INSERT INTO boreholes VALUES (2, 'Beta-02', 'Sedimentary');",
    "INSERT INTO boreholes VALUES (3, 'Gamma-03', 'Basalt');",
    
    "INSERT INTO seismic_readings VALUES (101, 1, 3.2, 5.5);",
    "INSERT INTO seismic_readings VALUES (102, 1, 4.1, 12.3);",
    "INSERT INTO seismic_readings VALUES (103, 2, 2.5, 4.1);",
    "INSERT INTO seismic_readings VALUES (104, 3, 5.0, 15.2);",
    "INSERT INTO seismic_readings VALUES (105, 1, 3.8, 8.9);"
]
#  TEXT is string, REAL is float; INTEGER REFERENCES boreholes(id) references to other table

# 3. Execute the Setup
with engine.connect() as conn:
    for statement in setup_statements:
        conn.execute(text(statement))
    conn.commit()
    print("Database created and populated successfully!")

# 4. Run a JOIN Query to prove it worked
# This joins the two tables so we can see the Borehole Name next to the Magnitude
query_sql = """
SELECT 
    b.name AS borehole_name, 
    b.formation, 
    s.magnitude, 
    s.depth_km
FROM seismic_readings s
JOIN boreholes b ON s.borehole_id = b.id
ORDER BY s.magnitude DESC;
"""
# Without a JOIN, your sensor data is just a bunch of numbers with no location context;
# with the JOIN, you have a complete geological picture.
with engine.connect() as conn:
    result = conn.execute(text(query_sql))
    df_0 = pd.DataFrame(result.mappings().all())

# 5. Show results
# Force the columns to be in the exact order you prefer
desired_order = ['borehole_name', 'formation', 'magnitude', 'depth_km']
df_0 = df_0[desired_order]

df_0

Database created and populated successfully!


,borehole_name,formation,magnitude,depth_km
0,Gamma-03,Basalt,5.0,15.2
1,Alpha-01,Basalt,4.1,12.3
2,Alpha-01,Basalt,3.8,8.9
3,Alpha-01,Basalt,3.2,5.5
4,Beta-02,Sedimentary,2.5,4.1


## Several ways to display tables

In [3]:
# Function to quickly view a table
def view_table(table_name, engine):
    with engine.connect() as conn:
        result = conn.execute(text(f"SELECT * FROM {table_name}"))
        df = pd.DataFrame(result.mappings().all())
    return df

# Display both tables
print("--- BOREHOLES TABLE ---")
display(view_table("boreholes", engine))

print("\n--- SEISMIC_READINGS TABLE ---")
display(view_table("seismic_readings", engine))

--- BOREHOLES TABLE ---


,formation,id,name
0,Basalt,1,Alpha-01
1,Sedimentary,2,Beta-02
2,Basalt,3,Gamma-03



--- SEISMIC_READINGS TABLE ---


,borehole_id,depth_km,magnitude,reading_id
0,1,5.5,3.2,101
1,1,12.3,4.1,102
2,2,4.1,2.5,103
3,3,15.2,5.0,104
4,1,8.9,3.8,105


In [31]:
# TABLES SIDE BY SIDE
from IPython.display import display, HTML

# 1. Get the data
with engine.connect() as conn:
    df1 = pd.DataFrame(conn.execute(text("SELECT * FROM boreholes")).mappings().all())
    df2 = pd.DataFrame(conn.execute(text("SELECT * FROM seismic_readings")).mappings().all())

# 2. Create the side-by-side HTML
html_str = f"""
<div style="display: flex; gap: 50px;">
    <div><h3>Boreholes</h3>{df1.to_html()}</div>
    <div><h3>Seismic Readings</h3>{df2.to_html()}</div>
</div>
"""
display(HTML(html_str))

print("\n--- JOIN borehole ---")
df_0


--- JOIN borehole ---


,borehole_name,formation,magnitude,depth_km
0,Gamma-03,Basalt,5.0,15.2
1,Alpha-01,Basalt,4.1,12.3
2,Alpha-01,Basalt,3.8,8.9
3,Alpha-01,Basalt,3.2,5.5
4,Beta-02,Sedimentary,2.5,4.1


In [29]:
# DISPLAY 3 TABLES SIDE BY SIDE
from IPython.display import display, HTML

master_query=query_sql

# 1. Fetch the data for all three tables
with engine.connect() as conn:
    df_boreholes = pd.DataFrame(conn.execute(text("SELECT * FROM boreholes")).mappings().all())
    df_readings = pd.DataFrame(conn.execute(text("SELECT * FROM seismic_readings")).mappings().all())
    
    # This is the "Result" table from our previous Master Query
    res = conn.execute(text(master_query))
    df_result = pd.DataFrame(res.mappings().all())
    
desired_order = ['borehole_name', 'formation', 'magnitude', 'depth_km']
df_result = df_result[desired_order]

# 2. Create the side-by-side HTML container
# We use 'flex-start' and 'overflow-x: auto' so it looks good even on smaller screens
html_layout = f"""
<div style="display: flex; flex-direction: row; gap: 20px; align-items: flex-start; overflow-x: auto;">
    <div style="flex: 1;">
        <h4 style="color: #2e76b1;">1. Boreholes (Metadata)</h4>
        {df_boreholes.to_html(index=False, classes='table table-striped')}
    </div>
    <div style="flex: 1;">
        <h4 style="color: #2e76b1;">2. Readings (Raw Data)</h4>
        {df_readings.to_html(index=False, classes='table table-striped')}
    </div>
    <div style="flex: 1;">
        <h4 style="color: #d1495b;">3. Final Query (Insight)</h4>
        {df_result.to_html(index=False, classes='table table-striped')}
    </div>
</div>
"""

display(HTML(html_layout))

## SQL (SQLAlchemy) stats: MAX, AVG

In [4]:
summary_query = """
SELECT 
    COUNT(*) AS total_readings,
    MAX(magnitude) AS max_magnitude,
    AVG(depth_km) AS average_depth
FROM seismic_readings;
"""

with engine.connect() as conn:
    result = conn.execute(text(summary_query))
    df_summary = pd.DataFrame(result.mappings().all())

df_summary

,average_depth,max_magnitude,total_readings
0,9.2,5.0,5


In [6]:
analysis_sql = """
SELECT 
    b.formation, 
    COUNT(s.reading_id) AS number_of_readings,
    ROUND(AVG(s.magnitude), 2) AS avg_magnitude,
    MAX(s.magnitude) AS max_magnitude
FROM boreholes b
JOIN seismic_readings s ON b.id = s.borehole_id
GROUP BY b.formation
ORDER BY avg_magnitude DESC;
"""

with engine.connect() as conn:
    result = conn.execute(text(analysis_sql))
    df_analysis = pd.DataFrame(result.mappings().all())

print("Geological Formation Seismic Summary:")
df_analysis

Geological Formation Seismic Summary:


,avg_magnitude,formation,max_magnitude,number_of_readings
0,4.03,Basalt,5.0,4
1,2.50,Sedimentary,2.5,1


## SQL (SQLAlchemy) HAVING

In [33]:
# Notice that we cannot use WHERE for 'avg_magnitude' 
# because 'avg_magnitude' doesn't exist until AFTER the grouping.
having_sql = """
SELECT 
    b.formation, 
    AVG(s.magnitude) AS avg_magnitude
FROM boreholes b
JOIN seismic_readings s ON b.id = s.borehole_id
GROUP BY b.formation
HAVING avg_magnitude > 3.5;
"""

with engine.connect() as conn:
    result = conn.execute(text(having_sql))
    df_high_activity = pd.DataFrame(result.mappings().all())

display(HTML(html_str)) # display original data

print("Formations with High Average Activity (> 3.5):")
df_high_activity

Formations with High Average Activity (> 3.5):


,avg_magnitude,formation
0,4.025,Basalt


JOIN, WHERE, GROUP BY, HAVING, and ORDER BY: 

In [32]:
master_query = """
SELECT 
    b.formation, 
    COUNT(s.reading_id) AS deep_reading_count,
    AVG(s.magnitude) AS avg_mag
FROM boreholes b
JOIN seismic_readings s ON b.id = s.borehole_id
WHERE s.depth_km > 5                  -- 1. Filter raw rows first
GROUP BY b.formation                  -- 2. Group them into buckets
HAVING avg_mag > 3.0                  -- 3. Filter the calculated results
ORDER BY avg_mag DESC;                -- 4. Sort the final output
"""

with engine.connect() as conn:
    result = conn.execute(text(master_query))
    df_master = pd.DataFrame(result.mappings().all())

display(HTML(html_str)) # display original data

df_master

,avg_mag,deep_reading_count,formation
0,4.025,4,Basalt


## SQL (SQLAlchemy) CREATE INDEX

In [12]:
# Create an index on the magnitude column
# This makes queries like "WHERE magnitude > 4.0" significantly faster
index_sql = "CREATE INDEX idx_magnitude ON seismic_readings(magnitude);"

with engine.connect() as conn:
    conn.execute(text(index_sql))
    conn.commit()
    print("Index 'idx_magnitude' created successfully.")

Index 'idx_magnitude' created successfully.


In [13]:
# Even though the SQL looks the same, the engine is now using the index 'under the hood'
indexed_filter_sql = """
SELECT * FROM seismic_readings 
WHERE magnitude BETWEEN 3.5 AND 5.0;
"""

with engine.connect() as conn:
    # Behind the scenes, SQLite sees 'magnitude' and jumps straight to the 
    # idx_magnitude index to find these rows instantly.
    result = conn.execute(text(indexed_filter_sql))
    df_filtered = pd.DataFrame(result.mappings().all())

df_filtered

,borehole_id,depth_km,magnitude,reading_id
0,1,8.9,3.8,105
1,1,12.3,4.1,102
2,3,15.2,5.0,104


In [14]:
with engine.connect() as conn:
    plan = conn.execute(text(f"EXPLAIN QUERY PLAN {indexed_filter_sql}"))
    for row in plan:
        print(f"Database Strategy: {row[3]}")

Database Strategy: SEARCH TABLE seismic_readings USING INDEX idx_magnitude (magnitude>? AND magnitude<?)
